In [3]:
import sys
import os

# Path to the folder containing ipyformkit (one level up from examples/)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root added to sys.path: {project_root}")
# Try importing now
import IpyFormKit as ifk

Project root added to sys.path: /Users/hoppe/Documents/Notebooks/IpyFormKit


In [4]:
test = {'first name':'Zaphod',
        'last name':'Beeblebrox',
        ('age', 'gender'):(-200, ('two-headed', 'male', 'female', 'non-binary')),
        'height':2.05,
        'date of birth':'Unknown (he probably lied about it anyway)',
        ('Input 1', 'Input 2', 'Input 3', 'Input 4'):('president', 'of', 'the', 'galaxy'),
        ('street', 'house'):('Heart of Gold Spaceship', 1),
        'additional info':{'hobbies':'Kidnapping himself, avoiding responsibility, being outrageous',}
       }


mandatory = {'first name': True, 'last name': True, 'age':True}
disable = {'last name': lambda d: d['first name'] == '',}
hide = {'house': lambda d: d['street'] == '',}
check = {'age': lambda d: d['age'] > 0,}

form = ifk.Form(test, title='Test Form', mandatory=mandatory, disable=disable, hide=hide, check=check, max_width=400)
form.display()

In [2]:
import ipyformkit as ifk

example_data = {
    'username': 'user123',
    'password': '',
    'confirm password': '',
    'email': 'user@example.com',
    ('subscribe to newsletter', 'newsletter frequency'): (True, ('Daily', 'Weekly', 'Monthly')),
    'address': '',
    ('city', 'state', 'zip code'): ('', '', ''),
    'additional info': {'comments': 'Enter your comments here...'},
}

disable_conditions = {
    'confirm password': lambda d: d['password'] == '',
    'newsletter frequency': lambda d: not d['subscribe to newsletter'],
}

hide_conditions = {
    'city': lambda d: d['address'] == '',
    'state': lambda d: d['address'] == '',
    'zip code': lambda d: d['address'] == '',
}

form = ifk.Form(
    example_data,
    title='User Registration Form',
    mandatory={'username': True, 'password': True, 'email': True},
    disable=disable_conditions,
    hide=hide_conditions,
)

form.display()

In [3]:
import ipyformkit as ifk
import ipywidgets as widgets
from IPython.display import display

form1 = ifk.Form(
    example_data,
    title='User Registration Form',
    mandatory={'username': True, 'password': True, 'email': True},
    disable=disable_conditions,
    hide=hide_conditions,
)

form2 = ifk.Form(
    example_data,
    title='User Registration Form'
)

wall = ifk.Masonry([form1, form2])
wall.display()

Box(children=(VBox(children=(Label(value='User Registration Form', _dom_classes=('ifk-form-title',)), HBox(chi…